In [3]:
# 可跑版 使用angle-bert-base-uncased-nli-en-v1进行single-pass  ByTitle
# update:加了评估，可对第一天结果进行评估与记录评估结果
# updata:ByTitle
# update：使用angle加载
# updata:to .py :single-pass-ByTitle-angle-bert-AngleLOAD-Eval.py
# update:保存predict_clusters
# update:3.6 load FIX data
# 暂时聚类最佳:angle-bert ANGLE_LOAD thresold = 0.975
# update：3.9 新增Score计算（以angle-bert ANGLE_LOAD thresold = 0.975为例）
import pandas as pd
# from sentence_transformers import SentenceTransformer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import torch
from transformers import AutoModel, AutoTokenizer
from angle_emb import AnglE


# yes! 聚类评估！！！可跑 TP, FP, TN, FN 得到RI、Precision、Recall、F1，ARI
# update:单个成簇的处理
from itertools import combinations
from math import comb


In [5]:
# 使用angle加载
model_id = '../NewsAthmTask2/models/angle-bert-base-uncased-nli-en-v1'
angle = AnglE.from_pretrained(model_id, pooling_strategy='cls_avg').cuda()

# 加载数据
# data = pd.read_csv('Data231202-231211.csv')
# data = pd.read_csv('./Data231202-231211_FIX/Data231202_newDATA.csv')
data = pd.read_csv('./Data231202-231211/Data231202.csv')


# 将日期转换为日期时间格式
data['pub_time'] = pd.to_datetime(data['pub_time'])

# 获取唯一日期列表
dates = data['pub_time'].dt.date.unique()



In [6]:
# 定义聚类中心更新函数
def update_cluster_center(cluster):
    # cluster_embeddings = sbert_model.encode(cluster)
    cluster_embeddings = angle.encode(cluster, to_numpy=True) # 使用angle加载
     
    return np.mean(cluster_embeddings, axis=0)

In [7]:
# 设置阈值
threshold = 0.972

# 对于每个日期
cluster_results = []
cnt = 0
for date in dates:
    print(cnt)
    cnt+=1
    # 获取该日期的新闻标题
    news_data = data[data['pub_time'].dt.date == date]['title'].tolist()
    # 获取该日期的新闻正文
    # news_data = data[data['pub_time'].dt.date == date]['body'].tolist() # ByBody

    # 使用SBERT模型获取语义向量
    # embeddings = sbert_model.encode(news_data)

    embeddings = angle.encode(news_data, to_numpy=True) # 使用angle加载

    # 定义当天的簇列表
    daily_clusters = []

    # 对于每个新闻数据
    # for i, embedding in enumerate(data_vec):
    for i, embedding in enumerate(embeddings):
        # 如果簇列表为空，则新开一个簇
        if not daily_clusters:
            # daily_clusters.append({'center': embedding, 'members': [news_data[i]]})
            daily_clusters.append({'center': embedding, 'members': [i],'news':[news_data[i]]}) # 改为存index
            continue

        # 计算当前数据点与各个簇中心的相似度
        # similarities = [cosine_similarity([embedding], [cluster['center']])[0][0] for cluster in daily_clusters]
        similarities = [cosine_similarity([embedding], [cluster['center']])[0][0] for cluster in daily_clusters]

        # 找到最大相似度及其对应的簇索引
        max_similarity = max(similarities)
        max_index = similarities.index(max_similarity)

        # 如果最大相似度大于阈值，则将当前数据点加入对应簇，并更新簇中心
        if max_similarity > threshold:
            # daily_clusters[max_index]['members'].append(news_data[i])
            daily_clusters[max_index]['members'].append(i) # 改为存index
            daily_clusters[max_index]['news'].append(news_data[i]) # 改为存index
            # daily_clusters[max_index]['center'] = update_cluster_center(daily_clusters[max_index]['members'])
            # daily_clusters[max_index]['center'] = update_cluster_center(daily_clusters[max_index]['members'],news_data)
            daily_clusters[max_index]['center'] = update_cluster_center(daily_clusters[max_index]['news'])
        # 否则新开一个簇
        else:
            # daily_clusters.append({'center': embedding, 'members': [news_data[i]]})
            daily_clusters.append({'center': embedding, 'members': [i],'news':[news_data[i]]}) # 改为存index

    # 将当天的簇信息添加到结果列表中
    cluster_results.append({'date': date, 'clusters': daily_clusters})

# 评估
#    true_clusters = [[0],[1],[2,16],[3],[4,6,22,50,73,87],[5],[7],[8,61],[9],[10,77],[11],[12],[13],
# [14,29,41,51,59,67,78,84],[15],[17],[18],[19],[20],[21,68],[23],[24],[25],[26],
# [27],[28],[30],[31],[32],[33],[34],[35,55],[36],[37],[38],[39],[40],[42],[43,64],
# [44],[45],[46],[47,53,88],[48],[49],[52],[54],[56],[57],[58],[60],[62],[63],[65],
# [66],[69],[70],[71],[72],[74],[75],[76],[79],[80],[81],[82],[83],[85],[86],
# [89],[90],[91],[92],[93],[94],[95]]

# update:3.6 新true_clusters for FIX
# true_clusters = [[0],[1,4,6,23,28,41],[2],[3],[5],[7],[8],[9,31],[10],[11],[12],[13],[14],[15],[16],[18],[19],[20],[17,21,38],[22],[24],[25],[26],[27],[29],[30],[32],[33],[34],[35],[36],[37],[39],[40],[42],[43],[44],[45],[46],[47],[48,49],[50],[51],[52],[53],[54],[55],[56],[57],[58],[59],[60],[61],[62],[63],[64],[65],[66],[67],[68,70,74],[69],[71],[72],[73],[75],[76],[77],[78],[79],[80]]


predicted_clusters = []
for cluster in cluster_results[0]['clusters']: # 2023-12-02的簇s
    clus_index = []
    for i in cluster['members']:
        clus_index.append(i)
    predicted_clusters.append(clus_index)
print(predicted_clusters)



0
[[0], [1], [2], [3], [4, 87], [5], [6, 23], [7], [8], [9], [10, 77], [11], [12], [13], [14], [15], [16], [17], [18], [19], [20], [21, 68], [22, 50], [24], [25], [26], [27], [28], [29, 78], [30], [31], [32], [33], [34], [35], [36], [37], [38], [39], [40, 43], [41, 59], [42], [44], [45], [46], [47, 88], [48], [49], [51, 67], [52], [53], [54], [55], [56], [57], [58], [60], [61], [62], [63], [64], [65], [66], [69], [70], [71], [72], [73], [74], [75], [76], [79], [80], [81], [82], [83], [84], [85], [86], [89], [90], [91], [92], [93], [94], [95]]


In [8]:
# 现在我有一个列表如下所示[[0], [1], [2], [3], [4, 87], [5], [6, 23], [7], [8], [9], [10, 77], [11], [12], [13], [14], [15], [16], [17], [18], [19], [20], [21, 68], [22, 50], [24], [25], [26], [27], [28], [29, 78], [30], [31], [32], [33], [34], [35], [36], [37], [38], [39], [40, 43], [41, 59], [42], [44], [45], [46], [47, 88], [48], [49], [51, 67], [52], [53], [54], [55], [56], [57], [58], [60], [61], [62], [63], [64], [65], [66], [69], [70], [71], [72], [73], [74], [75], [76], [79], [80], [81], [82], [83], [84], [85], [86], [89], [90], [91], [92], [93], [94], [95]]
# ，列表中每个子列表表示一个聚类的簇，子列表的元素个数即为簇中元素个数，然后我有一个语料文件'../Data231202-231211/Data231202.csv'，现在这个列表中的每个子列表的元素为该语料文件中的语料索引，现在我需要新增一列clus_news_num用来记录每个语料对应的簇中的元素个数，并将整个语料问价按语料对应的簇中的元素个数进行从大到小排序，要求同样大小的排名相同，然后在新增一列S_scale，该列内容为根据语料对应的簇中的元素个数进行归一化后的数，还有一列S_score为20*S_scale,还有一列index表示语料原始的坐标，最后将dataframe中的index，title,body,clus_news_num,S_scale，S_score这几列存入到新的csv文件中，请给出完整详细的代码
# update:3.9 可跑！！
from sklearn.preprocessing import MinMaxScaler

# 已有的聚类结果
# clusters = [[0], [1], [2], [3], [4, 87], [5], [6, 23], [7], [8], [9], [10, 77], [11], [12], [13], [14], [15], [16], [17], [18], [19], [20], [21, 68], [22, 50], [24], [25], [26], [27], [28], [29, 78], [30], [31], [32], [33], [34], [35], [36], [37], [38], [39], [40, 43], [41, 59], [42], [44], [45], [46], [47, 88], [48], [49], [51, 67], [52], [53], [54], [55], [56], [57], [58], [60], [61], [62], [63], [64], [65], [66], [69], [70], [71], [72], [73], [74], [75], [76], [79], [80], [81], [82], [83], [84], [85], [86], [89], [90], [91], [92], [93], [94], [95]]
clusters = predicted_clusters

# 创建一个字典，键是语料索引，值是对应的簇大小
index_to_cluster_size = {index: len(cluster) for cluster in clusters for index in cluster}

# 读取语料文件
df = pd.read_csv('./Data231202-231211/Data231202.csv')

# 新增列clus_news_num，记录每个语料对应的簇的大小
df['clus_news_num'] = df.index.map(index_to_cluster_size)

# 根据簇大小进行排序，并添加排名，相同大小的排名相同
df = df.sort_values(by='clus_news_num', ascending=False)
df['rank'] = df['clus_news_num'].rank(method='min', ascending=False)

# 新增列S_scale，为簇大小的归一化结果
scaler = MinMaxScaler()
df['S_scale'] = scaler.fit_transform(df[['clus_news_num']])

# 新增列S_score，为S_scale的值乘以20
df['S_score'] = df['S_scale'] * 20

# 新增列index，表示语料原始的坐标
df['ori_indexFrom0'] = df.index

# 只保留需要的列，并保存到新的CSV文件
final_df = df[['ori_indexFrom0', 'title', 'body', 'clus_news_num', 'rank','S_scale', 'S_score']]
final_df.to_csv('./T1ClusterScore/final_result.csv', index=False)
print("FINISH!")

FINISH!
